# KVecinos en un dataset de detección de spam

Se propone emplear un clasificador basado en distancias sobre el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


(4601, 57)


/home/yassin/anaconda3/envs/per/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## El clasificador por los vecinos más cercanos

In [2]:
from sklearn.neighbors import KNeighborsClassifier

kv = KNeighborsClassifier()
acc=kv.fit(X_train,y_train).score(X_test,y_test)

print(f'Precisión: {acc:.1%}')

Precisión: 79.5%


**Ejercicio:** Explora el principal parámetros del KNN (n_neighbors) y realiza una búsqueda mediante alguna técnica de optimización ya vista en la práctica anterior

In [3]:
# Solución

from sklearn.model_selection import GridSearchCV

G = {"n_neighbors":[1,3,4,5,10]}

GS = GridSearchCV(KNeighborsClassifier(), G, scoring='accuracy', refit=True, cv=5)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 82.1% con {'n_neighbors': 1}


## Mejoras

La función de distancia empleada por defecto es la distancia euclídea. Dicha distancia requiere un preproceso de las muestras para que tengan una escala similar todas ellas. Además KNN podría beneficiarse de una proyección mediante PCA con el fin de reducir la dimensionalidad.

**Ejercicio:** Implementa un pipeline con la normalización de los datos y un PCA, seguido del KNN. Busca los mejores parámetros. Se podría conseguir una tasa de acierto >90%.


In [4]:
# Solución

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline;

scaler = StandardScaler()
pca = PCA()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("knn", knn)])

G = {"pca__n_components": [5,10,15,20,25,50], "knn__n_neighbors": [1,3,4,5]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 90.8% con {'knn__n_neighbors': 3, 'pca__n_components': 20}


También podríamos probar diferentes funciones de distancia [sklearn distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics) a emplear en el parámetro "metric". Así mismo podríamos explorar el parámetro "weights" que pondera el voto de cada vecino de forma diferente según el parámetro escogido.

**Ejercicio:** prueba también diferentes métricas y "weights" junto con todo lo anterior. Emplea el BayessianOpt visto en la práctica anterior.

In [11]:
# Solución
!pip install scikit-optimize


In [5]:
# Solución
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV;
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;
from skopt import BayesSearchCV;


# Probar sólo 10 combinaciones de parámetros, n_iter=10
scaler = StandardScaler()
pca = PCA()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("knn", knn)])

G = {"pca__n_components": Integer(1,57),
     "knn__n_neighbors": Integer(1,20),
     "knn__metric": Categorical(["l1","l2"]),
     "knn__weights":Categorical(["uniform","distance"])}

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Precisión: 92.4% con OrderedDict([('knn__metric', 'l2'), ('knn__n_neighbors', 14), ('knn__weights', 'distance'), ('pca__n_components', 17)])


## Olivetti Faces

Prueba ahora el clasificador KNN junto con todos los parámetros y preprocesos que creas convenientes sobre el dataset de reconocimiento facial de Olivetti.

In [1]:
# Solución
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.datasets import fetch_olivetti_faces
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV;
from sklearn.decomposition import PCA;
from sklearn.pipeline import Pipeline;
from skopt import BayesSearchCV;

X, y = fetch_olivetti_faces(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


# Probar sólo 10 combinaciones de parámetros, n_iter=10
scaler = StandardScaler()
pca = PCA()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("knn", knn)])

G = {"pca__n_components": Integer(10,200),
     "knn__n_neighbors": Integer(1,10),
     "knn__metric": Categorical(["l1","l2"]),
     "knn__weights":Categorical(["uniform","distance"])}

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5,verbose=10)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

NameError: name 'fetch_olivetti_faces' is not defined